<a href="https://colab.research.google.com/github/Thunradee/COVID-19-Chest-X-Ray-Image-Classification/blob/main/AutoKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto-Keras

This file using Auto-Keras to classify COVID-19 from chest X-Ray images.
I preselected the images and upload to Google Drive for convenience purposes.The images are from the following:


*   https://github.com/ieee8023/covid-chestxray-dataset
*   https://www.kaggle.com/andrewmvd/convid19-x-rays?select=xrays.csv.
*   https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia.



## Install Packages

In [ ]:
pip install keras

In [ ]:
pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc1

In [ ]:
pip install autokeras

In [ ]:
pip install tensorflow

In [ ]:
pip install trains

## Program

In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
import keras
import tensorflow as tf
from keras import layers, models
from keras.models import load_model
from keras.preprocessing import image
from autokeras import StructuredDataClassifier, ImageClassifier
from sklearn.metrics import classification_report, plot_confusion_matrix
import os
from copy import deepcopy

In [6]:
# Import images from file
train_datagen = image.ImageDataGenerator(rescale = 1./255)
test_datagen = image.ImageDataGenerator(rescale = 1./255)

train = train_datagen.flow_from_directory(
    'drive/MyDrive/CptS-570-ML/Dataset/Train',
    target_size = (250, 250),
    batch_size = 1016,
    class_mode = 'binary'
)

test = test_datagen.flow_from_directory(
    'drive/MyDrive/CptS-570-ML/Dataset/Test',
    target_size = (250, 250),
    batch_size = 252,
    class_mode = 'binary'
)

Found 1016 images belonging to 2 classes.
Found 252 images belonging to 2 classes.


In [7]:
# Extract data into X and y
X_train, y_train = next(train)
X_train = deepcopy(X_train)
y_train = deepcopy(y_train)
y_train = y_train.astype(int)

X_test, y_test = next(test)
X_test = deepcopy(X_test)
y_test = deepcopy(y_test)
y_test = y_test.astype(int)

In [ ]:
# Training
# We limit max_trials and epochs due to the limitation of memory
ak = ImageClassifier(overwrite=True, max_trials=30)
ak.fit(x=X_train, y=y_train, epochs=100)

Trial 30 Complete [00h 03m 19s]
val_loss: 0.35257425904273987

Best val_loss So Far: 0.3243141770362854
Total elapsed time: 02h 36m 12s
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
Not enough memory, reduce batch size to 16.
Epoch 1/100
      2/Unknown - 0s 170ms/step - loss: 4.9872 - accuracy: 0.5000WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0245s vs `on_train_batch_end` time: 0.1517s). Check your callbacks.
64/64 [==============================] - 12s 192ms/step - loss: 0.7086 - accuracy: 0.7559
Epoch 2/100
64/64 [==============================] - 12s 191ms/step - loss: 0.2915 - accuracy: 0.8898
Epoch 3/100
64/64 [==============================] - 12s 192ms/step - loss: 0.1298 - accuracy: 0.9587
Epoch 4/100
64/64 [==============================] - 12s 191ms/step - loss: 0.0425 - accuracy: 0.9951
Epoch 5/100
64/64 [==============================] - 12s 191ms/step - loss: 0.0336 - accuracy: 0.9941
Epoch 6/100
64/64 [=

In [ ]:
# Evaluate the model
ak.evaluate(X_test, y_test)

8/8 [==============================] - 0s 38ms/step - loss: 0.5895 - accuracy: 0.8810


[0.5894883871078491, 0.8809523582458496]

In [ ]:
# Export the best performing model
model = ak.export_model()

# summarize the model
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 250, 250, 3)]     0         
_________________________________________________________________
cast_to_float32 (CastToFloat (None, 250, 250, 3)       0         
_________________________________________________________________
normalization (Normalization (None, 250, 250, 3)       7         
_________________________________________________________________
separable_conv2d (SeparableC (None, 248, 248, 32)      155       
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 246, 246, 64)      2400      
_________________________________________________________________
dropout (Dropout)            (None, 246, 246, 64)      0         
_________________________________________________________________
flatten (Flatten)            (None, 3873024)          

In [ ]:
# Save model
model.save("drive/MyDrive/CptS-570-ML/model_autokeras7", save_format="tf")

INFO:tensorflow:Assets written to: drive/MyDrive/CptS-570-ML/model_autokeras7/assets


In [ ]:
# The prediction are float and in an array
# So we convert it to a list of integers
preds = model.predict(X_test)
preds = np.rint(preds).astype(int)
preds = [p[0] for p in preds]

In [ ]:
# Calculate scores
report = classification_report(y_test, preds, target_names=['Negative', 'Positive'])
print(report)

              precision    recall  f1-score   support

    Negative       0.89      0.87      0.88       126
    Positive       0.88      0.89      0.88       126

    accuracy                           0.88       252
   macro avg       0.88      0.88      0.88       252
weighted avg       0.88      0.88      0.88       252



In [ ]:
# Confusion matrix
cm = tf.math.confusion_matrix(y_test, preds)
print(cm)

tf.Tensor(
[[110  16]
 [ 14 112]], shape=(2, 2), dtype=int32)


In [ ]:
# Calculate precision, recall and f-measure scores

TN = 110
FP = 16
FN = 14
TP = 112

ak_precision = TP/(TP+FP)
ak_recall = TP/(TP+FN)
ak_f = 2*(ak_precision*ak_recall)/(ak_precision+ak_recall)

print("Precision: {}".format(ak_precision))
print("Recall: {}".format(ak_recall))
print("F-measure: {}".format(ak_f))

Precision: 0.875
Recall: 0.8888888888888888
F-measure: 0.8818897637795274
